In [88]:
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import keras
import pandas as pd
from os import listdir
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split


In [4]:
path="/content/drive/MyDrive/Colab Notebooks/assignment/COVID_19_CT_Data"

In [86]:
train = pd.DataFrame(columns=['imageid','label'])


In [81]:
imageids=[]
labels=[]
path_tr = os.path.join(path,'Train')
path_te = os.path.join(path,'Test')
for dir in os.listdir(path_tr):
  for id in os.listdir(os.path.join(path,'Train',dir)):
    imageids.append(id)
    labels.append(dir) 
train.imageid=imageids
train['label']=labels



In [ ]:
count=0
for dir in os.listdir(os.path.join(path,'Train')) :
  for id in os.listdir(os.path.join(path,'Train',dir)) :
    img=cv2.imread(os.path.join(path,'Train',dir,id))
    cv2_imshow(img)
    count+=1
    if count>10 :
      break
baTch_size=32

In [60]:
#splitting the data into train & cv
train_data, val_data = train_test_split(train, test_size=0.15, random_state=42,stratify=train['label'])
print(train_data.shape)
print(val_data.shape)

(1190, 2)
(210, 2)


In [27]:
train_data['label'].value_counts()

non-COVID    595
COVID        595
Name: label, dtype: int64

In [89]:
test_ids = []
test_labels = []
for id in tqdm(listdir('Test/')):
    test_ids.append(id)
    if id.startswith('Covid'):
        test_labels.append(0) # This is how the data generator perceives COVID - 0, non-COVID - 1
    elif id.startswith('Non-Covid'):
        test_labels.append(1)


In [90]:
test= pd.DataFrame({'imageid': test_ids, 'label': test_labels})

In [61]:
train_ids = train_data.imageid
val_ids = val_data.imageid
train_labels = train_data.label
val_labels = train_data.label

In [30]:
import shutil 
for dir in os.listdir(path_tr):
  for id in os.listdir(os.path.join(path,'Train',dir)):
    shutil.copy(os.path.join(path, 'Train', dir, id), 'Train/'+id)
   

In [73]:
for dir in os.listdir(path_te):
  for id in os.listdir(os.path.join(path,'Test',dir)):
    shutil.copy(os.path.join(path, 'Test', dir, id), 'Test/'+id)

In [16]:
from tqdm.notebook import tqdm
for id in tqdm(val_ids):
  shutil.move('Train/'+id, 'Validation/'+id)


In [74]:
train_image_gen = ImageDataGenerator(rescale=1./255)
train_data_gen = train_image_gen.flow_from_dataframe(train_data, batch_size = baTch_size, x_col='imageid', y_col="label", class_mode="binary", target_size = (100, 100), 
                                                   directory = 'Train',)
val_data_gen = train_image_gen.flow_from_dataframe(val_data, batch_size = baTch_size, x_col='imageid', y_col="label", class_mode="binary", target_size= (100, 100), 
                                                   directory = 'Validation',)
test_data_gen = train_image_gen.flow_from_dataframe(test, batch_size = baTch_size, x_col='imageid', y_col="label", class_mode="binary", target_size = (100, 100), 
                                                   directory = 'Test',)


Found 1190 validated image filenames belonging to 2 classes.
Found 210 validated image filenames belonging to 2 classes.
Found 586 validated image filenames belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 1400 invalid image filename(s) in x_col="imageid". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [63]:

model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16,(3,3),activation='relu', input_shape=(100, 100, 3) ),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Dropout(0.4),

      tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Dropout(0.4),

      tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Dropout(0.4),

      tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),  

      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(512, activation='relu'),

      tf.keras.layers.Dense(1, activation='sigmoid')                                                   
])


In [64]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 98, 98, 16)        448       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 49, 49, 16)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 49, 49, 16)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 47, 47, 32)        4640      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 21, 21, 64)       

In [65]:
history = model.fit(train_data_gen, verbose = 1, epochs = 60, validation_data = val_data_gen )


Epoch 1/60
38/38 [==============================] - 2s 42ms/step - loss: 0.9056 - accuracy: 0.5160 - val_loss: 0.6827 - val_accuracy: 0.6238
Epoch 2/60
38/38 [==============================] - 1s 35ms/step - loss: 0.6290 - accuracy: 0.6222 - val_loss: 0.6091 - val_accuracy: 0.7429
Epoch 3/60
38/38 [==============================] - 1s 36ms/step - loss: 0.5498 - accuracy: 0.7268 - val_loss: 0.6058 - val_accuracy: 0.6905
Epoch 4/60
38/38 [==============================] - 1s 35ms/step - loss: 0.5221 - accuracy: 0.7453 - val_loss: 0.5821 - val_accuracy: 0.6667
Epoch 5/60
38/38 [==============================] - 1s 34ms/step - loss: 0.4723 - accuracy: 0.7853 - val_loss: 0.5107 - val_accuracy: 0.7619
Epoch 6/60
38/38 [==============================] - 1s 35ms/step - loss: 0.4507 - accuracy: 0.7915 - val_loss: 0.5693 - val_accuracy: 0.6905
Epoch 7/60
38/38 [==============================] - 1s 35ms/step - loss: 0.4903 - accuracy: 0.7495 - val_loss: 0.5198 - val_accuracy: 0.8000
Epoch 8/60
38

In [75]:
model.evaluate(test_data_gen)

19/19 [==============================] - 0s 22ms/step - loss: 1.6365 - accuracy: 0.6143


[1.636529803276062, 0.6143344640731812]

In [84]:
test.imageid.value_counts

<bound method IndexOpsMixin.value_counts of 0       Non-Covid (1009).png
1       Non-Covid (1021).png
2       Non-Covid (1018).png
3       Non-Covid (1005).png
4       Non-Covid (1001).png
                ...         
1981          Covid (83).png
1982          Covid (92).png
1983          Covid (91).png
1984          Covid (97).png
1985           Covid (8).png
Name: imageid, Length: 1986, dtype: object>

In [91]:
predictions = []
for id in tqdm(test.imageid.values):

    img = cv2.imread('/content/Test/' + id)
    X = np.empty((1,100,100,3),dtype=np.uint8)
    X[0,] = img
    predictions.append(int(model.predict(X)[0][0]))

In [92]:
from sklearn.metrics import confusion_matrix, classification_report
report = classification_report(test_labels, predictions)

In [93]:
print(report)

              precision    recall  f1-score   support

           0       0.52      0.98      0.68       302
           1       0.69      0.04      0.07       284

    accuracy                           0.53       586
   macro avg       0.60      0.51      0.38       586
weighted avg       0.60      0.53      0.39       586



In [94]:
confusion_matrix(test_labels, predictions)

array([[297,   5],
       [273,  11]])